In [3]:
%load_ext sql
%sql sqlite:////kaggle/input/storeee2222/store.db

Задание 1
Вывести страну, где популярнее всего группа Iron Maiden (т.е. треков куплено больше, чем в других странах). Также вывести кол-во купленных треков.

In [9]:
%%sql
SELECT c.Country, SUM(ii.Quantity) AS total_sales
FROM invoice_items ii
JOIN invoices i ON ii.invoiceid = i.invoiceid
JOIN customers c ON i.customerid = c.customerid
JOIN tracks t ON ii.trackid = t.trackid
JOIN albums al ON t.albumid = al.albumid
JOIN artists a ON al.artistid = a.artistid
WHERE a.name = 'Iron Maiden'
GROUP BY c.country
ORDER BY total_sales DESC
LIMIT 1;

 * sqlite:////kaggle/input/storeee2222/store.db
Done.


Country,total_sales
USA,61


Задание 2
Вывести "средний по длине" трек, среди приобретённых компаниями (только клиентами из конкретных компаний) после 22 апреля 2023 года. "Средний по длине" = максимально близкий к среднему значению множества треков по длине (не медиана!). Если таких треков несколько, вывести первый по алфавиту.

In [10]:
%%sql
WITH AverageLength AS (
    SELECT AVG(t.milliseconds / 1000.0) AS avg_length
    FROM invoice_items ii
    JOIN invoices i ON ii.invoiceid = i.invoiceid
    JOIN customers c ON i.customerid = c.customerid
    JOIN tracks t ON ii.trackid = t.trackid
    WHERE c.company IN ('BMW', 'JetBrains s.r.o.')
      AND i.invoicedate > '2023-04-22'
),
ClosestTrack AS (
    SELECT t.name, t.milliseconds / 1000.0 AS track_length,
           ABS(t.milliseconds / 1000.0 - (SELECT avg_length FROM AverageLength)) AS length_diff
    FROM invoice_items ii
    JOIN invoices i ON ii.invoiceid = i.invoiceid
    JOIN customers c ON i.customerid = c.customerid
    JOIN tracks t ON ii.trackid = t.trackid
    WHERE c.company IN ('BMW', 'JetBrains s.r.o.')
      AND i.invoicedate > '2023-04-22'
)
SELECT name, track_length
FROM ClosestTrack
ORDER BY length_diff ASC, name ASC
LIMIT 1;

 * sqlite:////kaggle/input/storeee2222/store.db
Done.


name,track_length
Riviera Paradise,528.692


Задание 3
Вывести треки, встречающиеся и в нескольких плейлистах, и в нескольких заказах одновременно.

In [11]:
%%sql
WITH TrackPlaylistCount AS (
    SELECT pt.trackid, COUNT(DISTINCT pt.playlistid) AS playlist_count
    FROM playlist_track pt
    GROUP BY pt.trackid
    HAVING COUNT(DISTINCT pt.playlistid) > 1
),
TrackOrderCount AS (
    SELECT ii.trackid, COUNT(DISTINCT i.invoiceid) AS order_count
    FROM invoice_items ii
    JOIN invoices i ON ii.invoiceid = i.invoiceid
    GROUP BY ii.trackid
    HAVING COUNT(DISTINCT i.invoiceid) > 1
)
SELECT t.name
FROM tracks t
JOIN TrackPlaylistCount plc ON t.trackid = plc.trackid
JOIN TrackOrderCount oc ON t.trackid = oc.trackid;

 * sqlite:////kaggle/input/storeee2222/store.db
Done.


Name
Sunshine Of Your Love
The Woman King
The Fix
Walkabout
Hot Girl
Gay Witch Hunt
Phyllis's Wedding
How to Stop an Exploding Man
Pilot


Задание 4
Вывести альбом, который является максимальным по продолжительности среди альбомов со стоимостью, большей средней цены за альбом (цена альбома = сумма цен треков, продолжительность считать аналогично).

In [12]:
%%sql
WITH AlbumPrices AS (
    SELECT a.albumid, SUM(t.unitprice) AS totalprice
    FROM albums a
    JOIN tracks t ON a.albumid = t.albumid
    GROUP BY a.albumid
),
AlbumDurations AS (
    SELECT a.albumid, SUM(t.milliseconds) AS totalduration
    FROM albums a
    JOIN tracks t ON a.albumid = t.albumid
    GROUP BY a.albumid
),
AveragePrice AS (
    SELECT AVG(totalprice) AS avg_price
    FROM AlbumPrices
)
SELECT ap.albumid, ad.totalduration
FROM AlbumPrices ap
JOIN AlbumDurations ad ON ap.albumid = ad.albumid
WHERE ap.totalprice > (SELECT avg_price FROM AveragePrice)
ORDER BY ad.totalduration DESC
LIMIT 1;

 * sqlite:////kaggle/input/storeee2222/store.db
Done.


albumid,totalduration
229,70665582


Задание 5
Для каждого исполнителя, у которого есть песни нескольких жанров, найти жанр, по которому он заработал наибольшее количество денег, а также вывести процентное соотношение суммы, заработанной исполнителем по этому жанру к общей сумме заработанных денег. Формат вывода: имя исполнителя, жанр, соотношение.

In [13]:
%%sql
WITH ArtistEarnings AS (
    SELECT a.artistid, a.name AS artistname, g.name AS genrename,
           SUM(t.unitprice * ii.quantity) AS genreearnings
    FROM artists a
    JOIN albums al ON a.artistid = al.artistid
    JOIN tracks t ON t.albumid = al.albumid
    JOIN invoice_items ii ON t.trackid = ii.trackid
    JOIN playlist_track pt ON t.trackid = pt.trackid
    JOIN genres g ON t.genreid = g.genreid
    GROUP BY a.artistid, g.genreid
),
TotalEarnings AS (
    SELECT artistid, SUM(genreearnings) AS totalearnings
    FROM ArtistEarnings
    GROUP BY artistid
),
MaxGenreEarnings AS (
    SELECT ae.artistname, ae.genrename, ae.genreearnings,
           te.totalearnings,
           (ae.genreearnings * 100.0 / te.totalearnings) AS percentage
    FROM ArtistEarnings ae
    JOIN TotalEarnings te ON ae.artistid = te.artistid
    WHERE ae.genreearnings = (
        SELECT MAX(genreearnings)
        FROM ArtistEarnings sub
        WHERE sub.artistid = ae.artistid
    )
)
SELECT artistname, genrename, percentage
FROM MaxGenreEarnings
WHERE artistname IN (
    SELECT artistname
    FROM ArtistEarnings
    GROUP BY artistname
    HAVING COUNT(DISTINCT genrename) > 1
);

 * sqlite:////kaggle/input/storeee2222/store.db
Done.


artistname,genrename,percentage
Guns N' Roses,Rock,58.346159227647966
Battlestar Galactica,TV Shows,47.368421052631575
Heroes,Drama,75.0
Lost,TV Shows,50.909090909090914
U2,Rock,66.66666666666667
The Office,TV Shows,53.96825396825397
